In [1]:
import torch
import numpy as np
from collections import OrderedDict
from graph.matrixview import render_incidence
from engine.torchengine import AnalyticalSetSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, FunctionSympy,get_analysis_structure, ElimResidual, ParallelResiduals, ParallelAnalysis, ipoptsolver
from engine.torchdata import load_file, process_expression, load_vals, generate_optim_functions, print_formatted_table, perturb
from engine.loaddata import process_json
from functools import partial
from scipy.optimize import minimize
from collections import namedtuple
import sympy as sp
import cyipopt
# Set the print options
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
data = load_file('testproblems')
problem_id = 0
fid, fdata = list(data.items())[problem_id]
polynomials, indices, edges, objective = process_json(fdata)
idxrev = {var.item():key for key,var in indices.items()}

sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(edges[1][idx][0],  rational=True) for idx,poly in polynomials.items()}
objective = FunctionSympy(objective, indices)

## Different ways to define orders

### A) Individual

In [3]:
analyses = {key: s.analysis for key,s in sets.items()}
residuals = {key: s.residual for key,s in sets.items()}

### 1) Pure

In [4]:
order = list(sorted(analyses.keys()))
A, B = [], order

### Option 1

In [50]:
# Merge residuals only
solvefor = torch.tensor([])
R = EliminateAnalysisMergeResiduals(functions=[residuals[idx] for idx in B])
P = EliminateAnalysis([analyses[idx] for idx in A], [objective,R])
objective_idx,residual_idx,equality_idx,inequality_idx = 0,1,None,None

### Option 3

In [17]:
solvefor = torch.tensor([])
T = ParallelResiduals([sets[idx].analysis for idx in B], [objective])
P = EliminateAnalysis([sets[idx].analysis for idx in A], [T], flatten=True)
objective_idx,residual_idx,equality_idx,inequality_idx = 0,None,1,None

### Initial guess

In [51]:
optim_indices = P.structure[0]

In [52]:
xs = torch.tensor([ 1.0000,  1.0000,  4.0000, -2.2670,  6.9916,  1.0000, -0.0420,  1.0000, 2.4799, -1.2697, -0.4501, -2.0834,  1.0000, -6.5000,  2.4820], dtype=torch.float64)

In [70]:
x0 = perturb(xs, 10, optim_indices, seed=12)

In [71]:
print_formatted_table([xs, x0], indices, idxrev, subset=optim_indices)

x_0   x_1    x_12    x_5     x_7     x_10   x_11    x_3     x_6    x_8    x_9     x_13    x_4    x_14    x_2   
1     1      1       1       1       -0.45  -2.083  -2.267  -0.042 2.48   -1.27   -6.5    6.992  2.482   4     
0.932 11.461 -18.108 -11.056 -12.207 -0.351 -12.415 -18.679 0.221  -0.192 -13.951 -16.059 120.17 -20.028 33.664


### Optimization

In [72]:
# MAKE SURE TO SET INEQUALITY DIRECTION, it is different for scipy and ipopt
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, hobj = generate_optim_functions(P,
    optim_indices, x0, inequality_direction='negative-null', 
    objective=objective_idx, residuals=residual_idx, equalities=equality_idx)
ineqlen, eqlen = len(ineq_function(xguess)), len(eq_function(xguess))
constraints = [{'type': 'eq', 'fun': eq_function, 'jac': deq}] if eqlen >= 1 else []
bnds_problem = [(None, None) for elt in optim_indices]

In [73]:
eq_function(xguess).numpy(), ineq_function(xguess).numpy(), obj_function(xguess)

(array([-10.94, -5.7, -34.63, 6.43, -12.52, 1269.7, -3.03, 2.97, -12.6,
        52.19]),
 array([], dtype=float32),
 794.3021034745311)

# IPOPT

In [74]:
def all_constraints(x):
    return np.concatenate([ineq_function(x), eq_function(x)])

def all_constraints_jac(x):
    if eqlen == 0:
        if ineqlen != 0:
            return dineq(x)
        else:
            return np.tensor([])
    elif ineqlen == 0:
        return deq(x)
    return np.concatenate([dineq(x), deq(x)], axis=0)

OptProblem = namedtuple('OptProblem', ['objective', 'constraints', 'gradient', 'jacobian', 'intermediate'])

lb,ub = zip(*bnds_problem)
cl = np.concatenate([-np.inf*np.ones(ineqlen), np.zeros(eqlen)])
cu = np.concatenate([np.zeros(ineqlen), np.zeros(eqlen)])

storeiter = [0]

def logiter(alg_mod, iter_count, obj_value, inf_pr, inf_du, mu, d_norm, regularization_size, alpha_du, alpha_pr, ls_trials):
    storeiter[0] = iter_count

# define the problem
probinfo = OptProblem(obj_function, all_constraints, dobj, all_constraints_jac, logiter)

prob = cyipopt.Problem(n=len(xguess), m=len(cu), lb=lb, ub=ub, cl=cl, cu=cu, 
                       problem_obj=probinfo)
prob.add_option('max_iter', 8000)
#prob.add_option('acceptable_tol', 1e-6)

In [75]:
xsol, optinfo = prob.solve(xguess)

In [76]:
storeiter

[75]

In [77]:
print_formatted_table([x0], indices, idxrev, subset=optim_indices)

x_0 x_1 x_12 x_5 x_7 x_10  x_11   x_3    x_6    x_8  x_9   x_13 x_4   x_14  x_2
1   1   1    1   1   -0.45 -2.083 -2.267 -0.042 2.48 -1.27 -6.5 6.992 2.482 4  


In [78]:
[(sets[elt].outputvar, sets[elt].expression) for elt in B]

[(x_11, 5*x_10*x_7/3 - 4/3),
 (x_8, 3*(-3*x_6 - 2)/(x_12*x_3)),
 (x_6, (5*x_0 - 6)/(9*x_11*x_9)),
 (x_13, -2*x_0*x_5 - 9/2),
 (x_4, (-x_8 - 2/3)/x_10),
 (x_14, 4*(-x_1*x_3*x_9 - 1)/(3*x_11)),
 (x_9, 4/(7*x_10)),
 (x_2, 4),
 (x_10, 3*(1 - 3*x_3)/(8*x_13)),
 (x_3, (3*x_14 + 2)/(2*x_11))]

In [79]:
print_formatted_table([sets[9].analysis(x0).detach().numpy()], indices, idxrev, subset=optim_indices)

x_0 x_1 x_12 x_5 x_7 x_10  x_11   x_3    x_6    x_8  x_9   x_13 x_4   x_14  x_2
1   1   1    1   1   -0.45 -2.083 -2.267 -0.042 2.48 -1.27 -6.5 6.992 2.482 4  


### Save results

In [82]:
import json

In [ ]:
name = 'results.json'
json_string = json.dumps(results, indent=4)
with open('../applications/data/{}'.format(name), 'w') as file:
    file.write(json_string)
